In [1]:
import pandas as pd

In [2]:
# read the purchase_data.json
jsonPath = "purchase_data2.json"
playerJson = pd.read_json(jsonPath)
playerJson.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


Total Number of Players

In [3]:
#Total Number of Players
dedupePlayer = pd.DataFrame(playerJson.groupby('SN').count())
totalNumPlayers =len(dedupePlayer)
totalNumPlayers_df = pd.DataFrame({"Total Players":totalNumPlayers},index = {0})
totalNumPlayers_df

,Total Players
0,74


Purchasing Analysis (Total)

In [4]:
# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

#Number of Unique Items
dedupeItem = pd.DataFrame(playerJson.groupby('Item ID').count())
totalUniqueItems = len(dedupeItem)

#Average Purchase Price
purchasePriceSeries = playerJson.loc[:,'Price']
avgPurchasePrice = round(purchasePriceSeries.mean(),2)

#Total Number of Purchases
numberPurchases = len(playerJson)

#Total Revenue
totalRevenue = round(playerJson.loc[:,'Price'].sum(),2)

#Purchasing Analysis Summary
purchasingAnalysisSummary_df = pd.DataFrame({"Num of Unique Items":totalUniqueItems,\
                                        "Avg Purchase Price":avgPurchasePrice,\
                                        "Total Purchases":numberPurchases,\
                                        "Total Revenue":totalRevenue},index ={0})
purchasingAnalysisSummary_df = purchasingAnalysisSummary_df[["Num of Unique Items",\
                                        "Avg Purchase Price",\
                                        "Total Purchases",\
                                        "Total Revenue"]]
purchasingAnalysisSummary_df

,Num of Unique Items,Avg Purchase Price,Total Purchases,Total Revenue
0,64,2.92,78,228.1


Gender Demographics

In [5]:
#Percentage and Count of Gender

#Dedupe the players
dedupeGender = playerJson.groupby(['SN','Gender']).count()

#Count by Gender
dedupeGender_df = pd.DataFrame(dedupeGender.groupby('Gender').count())
dedupeGenderCount = dedupeGender_df['Age'].rename("Count")

#Percentage by Gender
dedupeGenderPercentage = round(dedupeGenderCount/totalNumPlayers * 100,2).rename("Percentage")

#Percentage and Count of Gender Table
genderCountSummary_df = pd.DataFrame({"Percentage of Players":dedupeGenderPercentage,\
                                        "Total Count":dedupeGenderCount})
genderCountSummary_df = genderCountSummary_df[["Percentage of Players",\
                                        "Total Count"]]
genderCountSummary_df


,Percentage of Players,Total Count
Gender,,
Female,17.57,13
Male,81.08,60
Other / Non-Disclosed,1.35,1


Purchasing Analysis (Gender)

In [6]:
#Create a purchase table with Gender as groupby
genderGroupby = playerJson.groupby(['Gender'])

#Purchase Count
genderPurchaseCount = genderGroupby["Item ID"].count()

#Average Purchase Price
genderAvgPurchase = round(genderGroupby["Price"].mean(),2)

#Total Purchase Value
genderTotalPurchase = round(genderGroupby["Price"].sum(),2)

#Normalized Totals
genderNorPurchase = round(genderTotalPurchase/dedupeGenderCount,2)

#Gender Purchasing Summary
genderPurchaseSummary_df = pd.DataFrame({"Purchase Count":genderPurchaseCount,\
                                        "Avg Purchase Price":genderAvgPurchase,\
                                        "Total Purchase Value":genderTotalPurchase,\
                                        "Normalized Totals":genderNorPurchase})
genderPurchaseSummary_df = genderPurchaseSummary_df[["Purchase Count",\
                                                     "Avg Purchase Price",\
                                                     "Total Purchase Value",\
                                                     "Normalized Totals"]]
genderPurchaseSummary_df


,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,13,3.18,41.38,3.18
Male,64,2.88,184.60,3.08
Other / Non-Disclosed,1,2.12,2.12,2.12


In [7]:
#Age Demographics
#The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 

#Add an extra column to raw table with the bins for age
bins = [0, 10, 14, 19, 24, 29, 34, 39,playerJson["Age"].max()]
group_names = ["<10", "10-14", "15-19", "20-24","25-29","30-34","35-39","40+"]
playerJson["Age Bin"] = pd.cut( playerJson["Age"], bins, labels=group_names)
ageBinGroupby = playerJson.groupby(['Age Bin'])

#Purchase Count
agePurchaseCount = ageBinGroupby["Item ID"].count()

#Average Purchase Price
ageAvgPurchase = round(ageBinGroupby["Price"].mean(),2)

#Total Purchase Value
ageTotalPurchase = round(ageBinGroupby["Price"].sum(),2)

#Normalized Totals
#dedupe age using the player dedupe table
groupPlayerAge = playerJson.groupby(["SN","Age"]).count()
groupPlayerAge = groupPlayerAge.reset_index(level='Age', col_level=1)
groupPlayerAge["Age Bin"] =  pd.cut( groupPlayerAge["Age"], bins, labels=group_names)
groupAgeBin = groupPlayerAge.groupby(['Age Bin']).count()

ageNorPurchase = round(ageTotalPurchase/groupAgeBin['Age'],2)

#Age Demographic Summary
ageDemographicSummary_df = pd.DataFrame({"Purchase Count":agePurchaseCount,\
                                        "Avg Purchase Price":ageAvgPurchase,\
                                        "Total Purchase Value":ageTotalPurchase,\
                                        "Normalized Totals":ageNorPurchase})
ageDemographicSummary_df = ageDemographicSummary_df[["Purchase Count",\
                                                     "Avg Purchase Price",\
                                                     "Total Purchase Value",\
                                                     "Normalized Totals"]]
ageDemographicSummary_df


,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized Totals
Age Bin,,,,
<10,5,2.76,13.82,2.76
10-14,3,2.99,8.96,2.99
15-19,11,2.76,30.41,2.76
20-24,36,3.02,108.89,3.20
25-29,9,2.90,26.11,3.26
30-34,7,1.98,13.89,2.32
35-39,6,3.56,21.37,3.56
40+,1,4.65,4.65,4.65


Top Spenders

In [8]:
#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# SN, Purchase Count, Average Purchase Value, Total Purchase Value

#Create table that groupby SN
snGroupby = playerJson.groupby(['SN'])
snPurchaseCount = round(snGroupby["Item ID"].count(),2)
snAvgPurchase = round(snGroupby["Price"].mean(),2)
snTotalPurchase = round(snGroupby["Price"].sum(),2)
snSummary_df = pd.DataFrame({"Purchase Count":snPurchaseCount,\
                             "Average Purchase Value":snAvgPurchase,\
                             "Total Purchase Value":snTotalPurchase})

#Sort by total purchase value then take the top 5
snTop5Summary_df = snSummary_df.nlargest(5,'Total Purchase Value', keep='first')
snTop5Summary_df = snTop5Summary_df[["Purchase Count",\
                                    "Average Purchase Value",\
                                    "Total Purchase Value"]]
snTop5Summary_df

,Purchase Count,Average Purchase Value,Total Purchase Value
SN,,,
Sundaky74,2,3.70,7.41
Aidaira26,2,2.56,5.13
Eusty71,1,4.81,4.81
Chanirra64,1,4.78,4.78
Alarap40,1,4.71,4.71


Most Popular Items

In [9]:
#Identify the 5 most popular items by purchase count, then list (in a table):
# Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value

#Create table that groupby Item ID
itemGroupby = playerJson.groupby(['Item ID','Item Name', 'Price'])
itemPurchaseCount = round(itemGroupby["Price"].count(),2)
itemAvgPurchase = round(itemGroupby["Price"].mean(),2)
itemTotalPurchase = round(itemGroupby["Price"].sum(),2)
itemSummary_df = pd.DataFrame({"Purchase Count":itemPurchaseCount,\
                               "Total Purchase Value":itemTotalPurchase})

#Sort by total purchase count then take the top 5
itemTop5CountSummary_df = itemSummary_df.nlargest(5,'Purchase Count', keep='first')
itemTop5CountSummary_df = itemTop5CountSummary_df[["Purchase Count",\
                                    "Total Purchase Value"]]
itemTop5CountSummary_df

,,,Purchase Count,Total Purchase Value
Item ID,Item Name,Price,,
94,Mourning Blade,3.64,3,10.92
60,Wolf,2.70,2,5.40
64,Fusion Pummel,2.42,2,4.84
90,Betrayer,4.12,2,8.24
93,Apocalyptic Battlescythe,4.49,2,8.98


Most Profitable Items

In [10]:
#Identify the 5 most profitable items by total purchase value, then list (in a table):
# Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value
itemTop5ValueSummary_df = itemSummary_df.nlargest(5,'Total Purchase Value', keep='first')
itemTop5ValueSummary_df = itemTop5ValueSummary_df[["Purchase Count",\
                                    "Total Purchase Value"]]
itemTop5ValueSummary_df

,,,Purchase Count,Total Purchase Value
Item ID,Item Name,Price,,
94,Mourning Blade,3.64,3,10.92
117,"Heartstriker, Legacy of the Light",4.71,2,9.42
93,Apocalyptic Battlescythe,4.49,2,8.98
90,Betrayer,4.12,2,8.24
154,Feral Katana,4.11,2,8.22


Observations of three trends:
1. Males buy more products.
2. 20-24 age group buy the most products.
3. The frequency of purchases per user is low since the top user in purchase value only bought 2 times.